In [454]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
sns.set(style='dark', palette='muted', color_codes=True)
import matplotlib.pyplot as plt 
plt.style.use('ggplot')
import plotly.express as px 

In [455]:
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import LabelEncoder,OrdinalEncoder,OneHotEncoder,StandardScaler,MinMaxScaler 
from sklearn.compose import ColumnTransformer 
from sklearn.metrics import r2_score,mean_squared_error,silhouette_score,accuracy_score,confusion_matrix 
from sklearn.linear_model import LinearRegression,LogisticRegression 
from sklearn.ensemble import RandomForestRegressor 
from sklearn.cluster import KMeans 
from sklearn.decomposition import PCA 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

In [456]:
data = pd.read_csv(r'c:\Github\Sol\the_data\data-lab-5-hcv.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 615 entries, 0 to 614
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  615 non-null    int64  
 1   Category    615 non-null    object 
 2   Age         615 non-null    int64  
 3   Sex         615 non-null    object 
 4   ALB         614 non-null    float64
 5   ALP         597 non-null    float64
 6   ALT         614 non-null    float64
 7   AST         615 non-null    float64
 8   BIL         615 non-null    float64
 9   CHE         615 non-null    float64
 10  CHOL        605 non-null    float64
 11  CREA        615 non-null    float64
 12  GGT         615 non-null    float64
 13  PROT        614 non-null    float64
dtypes: float64(10), int64(2), object(2)
memory usage: 67.4+ KB


In [457]:
data.drop(columns=['Unnamed: 0'], inplace=True)

In [458]:
mis_val = data.isna().sum() * 100/len(data)

mis_val.sort_values(ascending=False)

ALP         2.926829
CHOL        1.626016
ALB         0.162602
ALT         0.162602
PROT        0.162602
Category    0.000000
Age         0.000000
Sex         0.000000
AST         0.000000
BIL         0.000000
CHE         0.000000
CREA        0.000000
GGT         0.000000
dtype: float64

In [459]:
data.duplicated().sum()
data.drop_duplicates(inplace=True, keep='last', subset=None, ignore_index=False)
data.duplicated().sum()

0

In [460]:
mean_imp = SimpleImputer(missing_values=np.NaN, strategy='mean')

data[['ALP', 'CHOL', 'ALB', 'ALT', 'PROT']] = mean_imp.fit_transform(data[['ALP', 'CHOL', 'ALB', 'ALT', 'PROT']] )

In [461]:
mis_val = data.isna().sum() * 100/len(data)

mis_val.sort_values(ascending=False)

Category    0.0
Age         0.0
Sex         0.0
ALB         0.0
ALP         0.0
ALT         0.0
AST         0.0
BIL         0.0
CHE         0.0
CHOL        0.0
CREA        0.0
GGT         0.0
PROT        0.0
dtype: float64

In [462]:
y = data.Category

le = LabelEncoder()



y_true = le.fit_transform(y)

data = pd.get_dummies(data, dtype=float)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 615 entries, 0 to 614
Data columns (total 18 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Age                              615 non-null    int64  
 1   ALB                              615 non-null    float64
 2   ALP                              615 non-null    float64
 3   ALT                              615 non-null    float64
 4   AST                              615 non-null    float64
 5   BIL                              615 non-null    float64
 6   CHE                              615 non-null    float64
 7   CHOL                             615 non-null    float64
 8   CREA                             615 non-null    float64
 9   GGT                              615 non-null    float64
 10  PROT                             615 non-null    float64
 11  Category_0=Blood Donor           615 non-null    float64
 12  Category_0s=suspect Bl

In [463]:
le.classes_ 

n_clusters = len(le.classes_)

In [464]:
preprocessing = Pipeline( 
    [ 
        ('scaler', MinMaxScaler()),
        ('pca', PCA(n_components=3, random_state=42))
    ]
)

In [465]:
clusterer = Pipeline( 
    [ 
        ( 
            'kmeans',
            KMeans( 
                n_clusters=n_clusters,
                init='k-means++',
                n_init=50,
                max_iter=500,
                random_state=42,
            )
        )
    ]
)

In [466]:
pipe = Pipeline( 
    [ 
        ('preprocessing', preprocessing),
        ('clusterer', clusterer)
    ]
)

In [467]:
pipe.fit(data)

Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('scaler', MinMaxScaler()),
                                 ('pca',
                                  PCA(n_components=3, random_state=42))])),
                ('clusterer',
                 Pipeline(steps=[('kmeans',
                                  KMeans(max_iter=500, n_clusters=5, n_init=50,
                                         random_state=42))]))])

In [468]:
preprocessed_data = pipe['preprocessing'].transform(data)
predicted_labels = pipe['clusterer']['kmeans'].labels_

In [469]:
mod_metric = silhouette_score(preprocessed_data,predicted_labels).round(3)
mod_metric

0.892

In [470]:
pcadf = pd.DataFrame(
     pipe["preprocessing"].transform(data),
    columns=["component_1", "component_2"],
 )

pcdaf['predicted_cluster'] = predicted_labels 
pcdaf['true_labels'] = le.inverse_transform(y_true)

f,ax = plt.subplots(figsize=(10,8))

ax.set_title('Cluster Matrix')

scat = sns.scatterplot( 
    data=pcdaf,
    s=300,
    x='component_1',
    y='component_2',
    hue='predicted_cluster',
    style='predicted_labels',
    palette='Set2'
)

scat.legend(bbox_to_anchor=(1.05,1), loc=2, borderaxespad=0.0)

ValueError: Shape of passed values is (615, 3), indices imply (615, 2)